In [31]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

print('Imports completed!')

Imports completed!


In [32]:
url = 'https://restcountries.com/v3.1/name/canada?fullText=true'
response = requests.get(url)
response.json()

[{'name': {'common': 'Canada',
   'official': 'Canada',
   'nativeName': {'eng': {'official': 'Canada', 'common': 'Canada'},
    'fra': {'official': 'Canada', 'common': 'Canada'}}},
  'tld': ['.ca'],
  'cca2': 'CA',
  'ccn3': '124',
  'cca3': 'CAN',
  'cioc': 'CAN',
  'independent': True,
  'status': 'officially-assigned',
  'unMember': True,
  'currencies': {'CAD': {'name': 'Canadian dollar', 'symbol': '$'}},
  'idd': {'root': '+1', 'suffixes': ['']},
  'capital': ['Ottawa'],
  'altSpellings': ['CA'],
  'region': 'Americas',
  'subregion': 'North America',
  'languages': {'eng': 'English', 'fra': 'French'},
  'translations': {'ara': {'official': 'كندا', 'common': 'كندا'},
   'bre': {'official': 'Kanada', 'common': 'Kanada'},
   'ces': {'official': 'Kanada', 'common': 'Kanada'},
   'cym': {'official': 'Canada', 'common': 'Canada'},
   'deu': {'official': 'Kanada', 'common': 'Kanada'},
   'est': {'official': 'Kanada', 'common': 'Kanada'},
   'fin': {'official': 'Kanada', 'common': 'Kana

In [33]:
# Basic structure of a data pipeline

def extract_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API request failed with status code: {response.status_code}")

url = 'https://restcountries.com/v3.1/name/canada'
raw_data = extract_data(url)
print(f"Extracted {len(raw_data)} records from API")

def transform_data(raw_data):
    pass

def load_data(transformed_data):
    pass

def run_pipeline():
    raw_data = extract_data()
    transformed_data = transform_data(raw_data)
    load_data(transform_data)

# run_pipeline()
    

Extracted 1 records from API


In [34]:
df = pd.DataFrame.from_dict(raw_data)
df.head()

,name,tld,cca2,ccn3,cca3,cioc,independent,status,unMember,currencies,...,gini,fifa,car,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,postalCode
0,"{'common': 'Canada', 'official': 'Canada', 'na...",[.ca],CA,124,CAN,CAN,True,officially-assigned,True,"{'CAD': {'name': 'Canadian dollar', 'symbol': ...",...,{'2017': 33.3},CAN,"{'signs': ['CDN'], 'side': 'right'}","[UTC-08:00, UTC-07:00, UTC-06:00, UTC-05:00, U...",[North America],"{'png': 'https://flagcdn.com/w320/ca.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [45.42, -75.7]}","{'format': '@#@ #@#', 'regex': '^([ABCEGHJKLMN..."


In [35]:
df.columns

Index(['name', 'tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status',
       'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region',
       'subregion', 'languages', 'translations', 'latlng', 'landlocked',
       'borders', 'area', 'demonyms', 'flag', 'maps', 'population', 'gini',
       'fifa', 'car', 'timezones', 'continents', 'flags', 'coatOfArms',
       'startOfWeek', 'capitalInfo', 'postalCode'],
      dtype='object')